<a href="https://colab.research.google.com/github/andrewdge/CSE354-Final-Project/blob/main/CSE354_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Document Level Predictions Using Paragraph Level Sentimient

Using the PerSent dataset, we will be training our model based on paragraph-level sentiments. These will be aggregated to produce document-level sentiments.

In [3]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 9.2 MB/s 
     |████████████████████████████████| 77 kB 5.5 MB/s 
     |████████████████████████████████| 596 kB 43.0 MB/s 
     |████████████████████████████████| 880 kB 45.4 MB/s 
     |████████████████████████████████| 6.6 MB 37.0 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=09481285118dde6438b6fd2f5dc33ad26784cf9c0f34a40dda2a99dba8b48c26
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [133]:
import torch
import math
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
import numpy as np
import os
from sklearn.metrics import precision_score, recall_score, f1_score
from transformers import DistilBertModel, DistilBertConfig, DistilBertTokenizer
torch.manual_seed(42)
np.random.seed(42)

# Constants

Constants we will use in our experiments. These may be subjected to change as hyperparameters

In [121]:
DISTILBERT_DROPOUT = 0.2
DISTILBERT_ATT_DROPOUT = 0.2
BATCH_SIZE = 16
EPOCHS = 3

# Andrew PATH
TEST_PATH = '/content/drive/MyDrive/CSE354/random_test.csv'
TRAIN_PATH = '/content/drive/MyDrive/CSE354/train.csv'

test = pd.read_csv(TEST_PATH)
train = pd.read_csv(TRAIN_PATH)



# Initializing Our Model

Here is where we set up our DistilBERT model.

In [ ]:
class DistillBERT():
  def __init__(self):
    # TODO(students): start
    self.tokenizer = DistilBertTokenizer.from_pretrained(model_name)
    config = DistilBertConfig(dropout=DISTILBERT_DROPOUT, 
                          attention_dropout=DISTILBERT_ATT_DROPOUT, 
                          output_hidden_states=True)
    model = DistilBertModel.from_pretrained("distilbert-base-uncased", config=config)
    # TODO(students): end
  def get_tokenizer_and_model(self):
    return self.model, self.tokenizer

# DataLoader

This class handles loading, preprocessing, and tokenizing the data.

Each row in the dataframe contains text with some number of paragraphs, as well as a number as labels per paragraph. We add another column in the dataframe, paragraphs per document. This will be used later to test our predictions as compare paragraph-level predictions to paragraph labels, as well as document-level predictions to document labels. We also remove data without paragraph-level labels.

For the labels create new columns for each.

This format is largely takes inspiration from Assignment 3.

In [134]:
class DatasetLoader(Dataset):
  def __init__(self, data, tokenizer):
    # Data is the uncleaned data, as a dataframe.
    self.data = data
    self.tokenizer = tokenizer

  def preprocess_data(self):
    # Combine labels into list.
    df = self.data
    df = df[df['Paragraph0'].notna()]
    df['Paragraph Labels'] = df.iloc[:, 5:].values.tolist() #Includes Nans, remove them
    self.data = df

  def tokenize_data(self):
    # Tokenizing
    tokens = []
    labels = []
    label_dict = {'Negative': 0,
                  'Neutral': 1,
                  'Positive': 2}
    document_list = self.data['DOCUMENT']
    label_list = self.data['Paragraph Labels']

    for (document, labels) in tqdm(zip(document_list, label_list), total=len(document_list)):
      paragraphs = document.split('\n')
      for paragraph, label in zip(paragraphs, labels):
        encoding = self.tokenizer(text=paragraph, truncation='longest_first', max_length=512, return_tensors='pt')
        labels.append(label)
        tokens.append(encoding.input_ids[0]) # Might need to CUDA
    
    tokens = pad_sequence(tokens, batch_first=True)
    labels = torch.tensor(labels)
    labels.to("cuda:0" if torch.cuda.is_available() else "cpu")
    tokens.to("cuda:0" if torch.cuda.is_available() else "cpu")
    dataset = TensorDataset(tokens, labels)
    return dataset

  def get_data_loaders(self, shuffle=True):
    processed_dataset = self.tokenize_data()
    data_loader = DataLoader(
        processed_dataset,
        shuffle=shuffle,
        batch_size=BATCH_SIZE
    )
    return data_loader

In [132]:
p = (train['DOCUMENT'].values[0])
x = p.split('\n')
print(len(x))

5
